In [1]:
# Install tensorflow version 1.13.1
%tensorflow_version 1.13.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
#Install packages required
!pip -qqq install gensim
!pip -qqq install git+https://www.github.com/keras-team/keras-contrib.git
!pip -qqq install --upgrade keras==2.2.4
!pip install sklearn-crfsuite
!pip -q install seqeval
!pip install sklearn-crfsuite

     |████████████████████████████████| 317kB 2.9MB/s 
     |████████████████████████████████| 747kB 4.1MB/s 
     |████████████████████████████████| 51kB 1.7MB/s 


In [3]:
#Check tensorflow version
import tensorflow as tf

print(tf.__version__)

1.15.2


In [4]:
#Import all required packages
import pandas as pd
import json

from tensorflow               import keras
from sklearn.model_selection  import train_test_split
from sklearn_crfsuite         import CRF #from keras_contrib.layers import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [5]:
#Download data from Kaggle
!mkdir ~/.kaggle
token = {"username":"gugankailasam","key":"7be55bcc2396ad6d025f6b7e2b3b53b0"}

with open('kaggle.json','w') as file:
  json.dump(token,file)

!cp ./kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d abhinavwalia95/entity-annotated-corpus
import zipfile
with zipfile.ZipFile("entity-annotated-corpus.zip","r") as zip_ref:
    zip_ref.extractall("data")
!ls

 64% 17.0M/26.4M [00:00<00:00, 45.6MB/s]
100% 26.4M/26.4M [00:00<00:00, 58.7MB/s]
data  entity-annotated-corpus.zip  kaggle.json	sample_data


In [6]:
df = pd.read_csv('/content/data/ner_dataset.csv', encoding= 'unicode_escape')#data//ner_dataset.csv
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [7]:
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048575,1048575,1048575
unique,47959,35178,42,17
top,Sentence: 30149,the,NN,O
freq,1,52573,145807,887908


In [8]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [9]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

In [10]:
df = df.fillna(method = 'ffill')

In [11]:
# This is a class that get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [12]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [13]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [14]:
sentences = getter.sentences

#Feature Preparation

In [15]:
#Methods which returns features, labels and tokens for given sentence and word. 
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [16]:
#Store features and labels of a sentence
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [17]:
#Split data for train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [18]:
#Model building and training
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)         
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [19]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

#Evaluating the model performance.

In [20]:
#Outputting the accuracy
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.971983668129485


In [21]:
#Analysing precision, recall, f1-score and support for each class
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-art       0.48      0.14      0.22        85
       B-eve       0.52      0.48      0.50        46
       B-geo       0.87      0.91      0.89      7546
       B-gpe       0.97      0.94      0.96      3162
       B-nat       0.77      0.35      0.49        48
       B-org       0.81      0.74      0.77      4021
       B-per       0.85      0.83      0.84      3343
       B-tim       0.92      0.87      0.89      4127
       I-art       0.10      0.02      0.03        52
       I-eve       0.27      0.32      0.30        28
       I-geo       0.82      0.82      0.82      1483
       I-gpe       0.88      0.67      0.76        45
       I-nat       0.60      0.25      0.35        12
       I-org       0.81      0.80      0.81      3282
       I-per       0.85      0.90      0.87      3400
       I-tim       0.84      0.73      0.78      1316
           O       0.99      0.99      0.99    178319

    accuracy              